In [ ]:
!pip install --upgrade boto3 -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install --upgrade sagemaker -i https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
# For notebook instances (Amazon Linux)
!sudo yum update -y
!sudo yum install amazon-linux-extras
!sudo amazon-linux-extras install epel -y
!sudo yum update -y
!sudo yum install git-lfs git -y

In [ ]:
#下载模型snapshot到本地，需要25G空间
#需大约15-30分钟时间，请耐心等待, 如果左侧大括号内还是[*]，就还在下载中，*变成任意数例如[3]就证明已完成

from pathlib import Path
local_model_path = Path("./chatglm3-6b")
local_model_path.mkdir(exist_ok=True)
model_name = "ZhipuAI/chatglm3-6b"
clone_path = f"https://www.wisemodel.cn/{model_name}.git"
print(clone_path)

!git lfs install
!git clone $clone_path

In [ ]:
#定义本地路径model_snapshot_path，以及s3路径s3_location，方便下一步从本地将模型snapshot上传到s3

import sagemaker

sagemaker_session = sagemaker.Session()
sagemaker_session_bucket = sagemaker_session.default_bucket()
s3_location = f"s3://{sagemaker_session_bucket}/{model_name.split('/')[-1]}/"

#你也可以把local_model_path直接替换成你的模型路径，例"model_snapshot_path=./chatglm3-6b", 这个文件夹里需要包含config.json
model_snapshot_path = local_model_path

print(f"model_snapshot_path: {model_snapshot_path}")
print("s3_location:",s3_location)

In [ ]:
#上传模型
!aws s3 sync $model_snapshot_path $s3_location

In [ ]:
#将模型的s3路径更新到inference.py中
!sed -i 's|s3_location =.*|s3_location = \"{s3_location}\"|' code/inference.py

In [ ]:
#开始部署

import boto3
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

print(role)
print(bucket)

if "cn-" in region_name:
    with open('./code/requirements.txt', 'r') as original: data = original.read()
    with open('./code/requirements.txt', 'w') as modified: modified.write("-i https://pypi.tuna.tsinghua.edu.cn/simple\n" + data)

!touch dummy
!tar czvf model.tar.gz dummy
assets_dir = 's3://{0}/{1}/assets/'.format(bucket, 'llm_chinese')
model_data = 's3://{0}/{1}/assets/model.tar.gz'.format(bucket, 'llm_chinese')
!aws s3 cp model.tar.gz $assets_dir
!rm -f dummy model.tar.gz

model_name = None
entry_point = 'inference.py'
framework_version = '1.13.1'
py_version = 'py39'
model_environment = {
    'SAGEMAKER_MODEL_SERVER_TIMEOUT':'600', 
    'SAGEMAKER_MODEL_SERVER_WORKERS': '1', 
}

from sagemaker.pytorch.model import PyTorchModel

model = PyTorchModel(
    name = model_name,
    model_data = model_data,
    entry_point = entry_point,
    source_dir = './code',
    role = role,
    framework_version = framework_version, 
    py_version = py_version,
    env = model_environment
)

endpoint_name = 'pytorch-inference-llm-v1'
# instance_type = 'ml.p3.2xlarge'
instance_type='ml.g4dn.12xlarge' 

instance_count = 1

from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type, 
    initial_instance_count = instance_count,
    serializer = JSONSerializer(),
    deserializer = JSONDeserializer()
)

### 测试


In [ ]:
#休眠5分钟,确保模型可以完全加载
import time
time.sleep(300)

In [ ]:
inputs= {
    "ask": "你好!"

}

response = predictor.predict(inputs)
print(response["answer"])

inputs= {
    "ask": "晚上睡不着应该怎么办"

}

response = predictor.predict(inputs)
print(response["answer"])

In [ ]:
inputs= {
    "ask": """Please provide an analysis and summary of the background, methods, and conclusions of the paper based on the overview below.
Desired context：
Background: <Background>
Methods: <Methods>
Conclusions: <Conclusions>

The following is an overview of the paper:
 
The study is a multicentered retrospective cohort study analyzing 1170 single frozen-thawed euploid blastocyst transfers following trophectoderm biopsy and preimplantation genetic testing (PGT) between May 2015 and February 2019. The study aimed to determine if there is an optimal window of time for transferring single frozen-thawed euploid blastocysts associated with a maximal live birth rate (LBR).
 
The results showed that performing a single frozen-thawed euploid blastocyst transfer at 160 §4 h post-hCG trigger in modified-natural frozen-thawed embryo transfer (FET) cycles was independently associated with a higher LBR as compared to transfers outside this window. However, in natural FET cycles, LBRs were comparable across a wider range of time intervals. The study also found that there was compelling evidence for maintaining embryo-endometrial synchrony to optimize clinical outcomes following FETs, which could potentially be achieved by matching the transfer time of an embryo post-ovulation to its developmental age post-oocyte retrieval.This study examines the impact of various factors, including cycle type (natural versus modified-natural), embryo quality (top versus good quality), embryo stage (fully hatched versus hatching or earlier blastocyst), vitriﬁcation day (D5 versus D6), and survival rate (>90% versus <90%), on the likelihood of successful embryo transfers. The results show that overall, embryo transfers conducted 160 days post-hCG trigger or LH surge on a modiﬁed-natural cycle are associated with a significantly higher likelihood of a higher live birth rate (LR 1.21, 95% CI 1.15-1.99) compared to the natural cycle group (LR 36.8%, 95% CI 33.7-39.9). The results suggest that there is potential to further optimize the timing of natural cycle embryo transfers. However, the study is limited by the presence of residual unknown bias and the results may not be applicable to general populations or embryos that have not undergone embryo biopsy.In 2016, the rate of embryo transfers following PGT-A and subsequent single or multiple embryo transfers in Australia and New Zealand was 36.9%. These rates are still suboptimal, and other embryonic or extra-embryonic factors must be considered. There is compelling evidence for maintaining embryo-endometrial synchrony to optimize clinical outcomes following an embryo transfer. This is true both in "fresh" cycles and frozen-thawed embryo transfer (FETs) cycles. A recently published review including hundreds of transfers across Europe and the USA has shown that the embryo implantation rate is significantly reduced when the embryo and endometrium are asynchronous by greater than 1.5 days. The main reason behind this is the existence of a clinical "window of implantation" where it has been proposed that the endometrium is most receptive to embryo implantation. To synchronize embryo transfers with this window, it seems physiologically intuitive to replace frozen-thawed blastocysts when their developmental age post-oocyte retrieval and the replacement cycle time post-ovulation match (i.e., around 120 h for full blastocysts or more advanced). However, it has been suggested that the implantation window is wide or can shift due to individual variations in endometrial pinopode expression. Therefore, earlier or later transfers can lead to comparable probabilities of live birth. Whether restricting blastocyst transfers to a specific interval of time around this 120 h post-ovulation period is beneficial has not yet been thoroughly investigated. Ovulation after the hCG trigger in modified-natural cycles is widely accepted to occur after /C2440 h (Fischer et al. 1993; Andersen et al. 1995). On the other hand, ovulation can occur 24-56 h after the spontaneous LH surge onset in natural cycles (Godbert et al. 2015; World Health Organization 1980), which rely on monitoring and interpretation of serum measurements that remain difficult to precisely define (Mackens et al. 2017). The aim of this study is to answer this highly relevant clinical question by evaluating whether there is an optimal time window for transferring single frozen-thawed euploid blastocysts in natural or modified-natural cycles, where LBRs are maximized.This study examines the effect of natural cycles on the timing of embryo transfers on the odds of live birth (LBR) in women. The study used hormonal criteria to determine the optimal timing of frozen euploid blastocyst transfers, which is 160 seconds after trigger/LH surge onset, or up to 120 seconds post-ovulation if the embryo transfer is more advanced. The study identified two groups: optimal timing (OT) where the transfer is conducted 160 seconds after trigger/LH surge onset, and non-optimal timing (NOT) where the transfer is conducted outside of this window. The study also analyzed the outcome measures of LBR, clinical pregnancy rates, and miscarriage rates. All continuous variables were expressed as mean with standard deviation, and all categorical variables were expressed as percentages. The study used generalized estimating equations (GEE) to account for any auto-correlation between the data due to clustering effect. The study identified the associations of variables such as maternal age, indication for PGT, FET cycle characteristics, and embryo characteristics with LBRs. The study also evaluated the potential confounders and performed a sensitivity analysis to check the robustness of the results.This study compares the effect size between modified-natural and natural groups in terms of overall and endometrial preparation protocols for advanced maternal age, doctor/patient decision, recurrent implantation failure (RIF), recurrent miscarriage, previous chromosomal abnormal conception, monogenic disorder, translocation or other reasons. The study analyzed data from 820 patients who underwent 933 stimulated cycles with PGT for advanced maternal age and 1170 single frozen-thawed euploid blastocyst transfers. The results showed that overall, 802/1170 cycles utilized luteal phase support, with the mean LBR in all cycles being 36.2%. The mean time from ovulation to embryo transfer in this study was 104.9 h. The study also found that there was no significant difference in patient age or indication for PGT between the optimal time window and non-optimal time window for embryo transfers. The duration of follicular phase and endometrial thickness before ovulation were similar across the optimal and non-optimal transfer cycles. The embryos transferred in the optimal time window showed no significant difference in embryo quality prior to cryopreservation, vitrification day, hatching status at vitrification or post-thaw cell survival rate.This study evaluates the effect of Ovulation Time (OT) on the probability of live birth in women undergoing preimplantation genetic testing (PGT). The study controlled for confounders such as modified natural and natural cycles, embryo quality, hatching or earlier stage, vitrification day, and survival rate. The results showed that OT was positively associated with live birth rate (RR 1.21, 95% CI 1.04–1.41) in single frozen-thawed euploid blastocyst transfers. The positive association between CPR (Chrono-Progesterone Ratio) and OT persisted but became non-statistically significant (RR 1.13, 95% CI 0.98–1.30), while the lower incidence of miscarriages associated with OT also persisted, remaining also statistically non-significant (RR 0.70, 95% CI 0.33–1.45). The study also evaluated the baseline patient and transfer cycle characteristics potentially associated with live birth.This text describes a study comparing the likelihood of genetic abnormalities in embryos transferred under optimal and non-optimal timings. The study found that embryo transfers under optimal timing resulted in significantly higher likelihood of genetic abnormalities compared to those under non-optimal timing. The study also found that the likelihood of genetic abnormalities varied based on the type and timing of luteal phase support and the average time from LH surge onset or trigger to transfer.This study compares the live birth rates (LBR) associated with transferring a single frozen-thawed euploid blastocyst at different time intervals after hCG trigger versus transferring at the same time intervals outside the window. The results show that transferring the blastocyst at the optimal timing (160 §4 h after hCG trigger) is associated with a higher LBR compared to transferring at other times. The study also found that transferring embryos within the modified-natural cycles (160 §6,§8a nd§12 h) was also associated with higher LBRs than transferring outside of this window. However, the optimal window could not be determined in the natural group due to a larger sample size.In 2019, a study found that the timing of embryo transfers is often based on various interpretations of serum LH, E2, and P4 level interactions to determine the onset of the LH surge, which is commonly accepted to occur 24 hours post-trigger. However, the study argues that relying on the value of LH alone may not be the optimal method as certain women may present with atypical LH kinetics. The study found that traditional methods of embryo transfer, which are often timed based on these interpretations, are complicated by differing kinetics between individuals, including the rate of LH rise and progesterone exposure, which may cause variations in the secretory transformation of the endometrium and the appearance of the implantation window. The study recommends using hCG, which provides a more robust determination of the exact time when lutealization begins and subsequent ovulation, which is commonly accepted to occur 24 hours post-trigger.This study compares the pregnancy rates and implantation rates between cycles with rising or falling progesterone levels and a well-documented LH surge. The study found that pregnancy rates with a well-documented LH surge are excellent and there is no significant difference in implantation rate or pregnancy rate between cycles with or without rising progesterone. However, this study is small (n ¼54) and therefore may not be powered to detect clinically meaningful differences. The current study utilizes hormonal criteria based on the interpretation of LH and P4 values observed in Hoff et al.'s (1983) study. This reveals comparable LBRs for transfers conducted across a range of post-surge times. The interpretation of this finding through the lens of embryo-endometrial synchrony is complicated by variations in time from the LH surge to ovulation between individuals. The study suggests that there may be further room to optimize natural cycle embryo transfers by improving the interpretation of markers of the LH surge and the exact time of ovulation to achieve embryo-endometrial synchrony.This study examines the association between live birth rates and different窗口 times after laparoscopic surgery for post-LH embryo transfer. The study controlled for survival rate, hatching stage, embryo quality, and vitrification day. The results show that the optimal timing of frozen euploid blastocyst transfers is not as narrow as previously thought and is associated with higher live birth rates. The study has limitations such as a small sample size and the retrospective nature of the study, but the multivariate regression analysis and sensitivity analyses did not significantly modify the findings.This study examines the associations between the timing of the LH surge onset and the LBRs (Live-Donor Blastocyst Transfer) in frozen-thawed euploid blastocyst transfers. The study only included PGT cycles, so the findings may not reflect trends in the general population. The study's criteria for the LH surge onset were based on previously published data, but the findings still persist even when accounting for the year of biopsy and FET cycle. The time intervals between the LH surge onset and embryo transfer for the included FET cycles varied widely, and the clinic's main transfer days were Monday, Wednesday, Friday, and Saturday. The study challenges the current dogma of determining the timing of embryo transfers in natural cycles using the LH surge onset and calls for more research to identify the markers for the optimal window within these cycles. The study also suggests that variations in transfer time of up to 1 day early or late can still potentially produce acceptable LBRs.This text describes a research study that analyzed the timing of human chorionic gonadotrophin (HCG) injections and follicular rupture. The study was designed and performed by B.A., who also revised the manuscript for important intellectual content. L.T. and M.C. approved the protocol and revised the manuscript for important intellectual content. The study was funded by the National Health and Medical Research Council Early Career Fellowship (GNT1147154). No other funding was received for this study. The authors did not disclose any conflicts of interest. The study compared the different methods used to determine the onset of the LH surge in urine during the human menstrual cycle.This text discusses various factors related to uterine receptivity and the timing of ovulation in cycling women receiving exogenous oestradiol and progesterone. The authors focus on the endometrial receptivity array for diagnosis and personalized embryo transfer as a treatment for patients with repeated implantation failure, as well as the effects of progesterone elevation on live birth rates after in vitro fertilization (IVF).
 
The first two sections of the text describe studies that investigated the endometrial receptivity of women, with a focus on the "nidation window" and the effects of nidation on fertility. The authors also mention a study that analyzed the endometrial receptivity array for personalized embryo transfer in advanced maternal age.
 
The next section discusses a randomized controlled study that evaluated the effects of preimplantation genetic diagnosis for aneuploidies in advanced maternal age using in vitro fertilization with preimplantation genetic diagnosis.
 
The following section describes a review of studies that investigated the effects of progesterone elevation on the day of hCG and live birth rates after IVF.
 
Finally, the authors discuss a study that investigated the optimal timing of frozen embryo transfer in natural cycles based on monitoring the ovulatory surge."""

}

response = predictor.predict(inputs)
print(response["answer"])

### 删除SageMaker  Endpoint
删除推理服务

In [ ]:
predictor.delete_endpoint()